In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/forest-cover-type-prediction/train.csv")
test_data = pd.read_csv("/kaggle/input/forest-cover-type-prediction/test.csv")

In [ ]:
data.head()

In [ ]:
data["Cover_Type"].value_counts()

In [ ]:
plt.bar(data.Cover_Type,data.Slope)

In [ ]:
correlation = data.corr()
correlation["Cover_Type"].sort_values(ascending=False)


In [ ]:
plt.bar(data["Cover_Type"],data['Horizontal_Distance_To_Fire_Points'])

In [ ]:
px.scatter_3d(x=data["Horizontal_Distance_To_Hydrology"],y=data["Vertical_Distance_To_Hydrology"],z=data["Elevation"])

In [ ]:
X = data.drop("Cover_Type",axis=1)
y = data["Cover_Type"]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_pca=pca.fit_transform(X)
px.scatter_3d(x=X_pca[:,0],y=X_pca[:,1],z=X_pca[:,2])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()

tree_clf.fit(X_train,y_train)
print(tree_clf.score(X_train,y_train))
print(tree_clf.score(X_test,y_test))

In [ ]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(X_train,y_train)
svc.score(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=50)

forest_clf.fit(X_train,y_train)
print(forest_clf.score(X_train,y_train))
print(forest_clf.score(X_test,y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
print(knn.score(X_train,y_train))
print(knn.score(X_test,y_test))

In [ ]:
from sklearn.model_selection import GridSearchCV
params1 = [{"n_estimators":[20,30,40,50]}]
params2 = [{"n_neighbors":[3,5,7,9],"algorithm":["brute","auto"]}]
params3 = [{"n_estimators":[20,30,40,50]}]


grid_tree = GridSearchCV(forest_clf,params1)
grid_knn = GridSearchCV(knn,params2)
grid_forest = GridSearchCV(forest_clf,params3)

In [ ]:
for i in [grid_knn,grid_tree,grid_forest]:
    i.fit(X_train,y_train)
    print(i.best_estimator_)
    print(i.best_score_)


In [ ]:
from sklearn.ensemble import VotingClassifier
vot_clf_2 = VotingClassifier(estimators=[("tree",grid_tree.best_estimator_),("knn",grid_knn.best_estimator_),("forest_clf",grid_forest.best_estimator_)],voting="hard")

vot_clf_2.fit(X_train,y_train)
print(vot_clf_2.score(X_train,y_train))
print(vot_clf_2.score(X_test,y_test))

In [ ]:
prediction = vot_clf_2.predict(test_data)

In [ ]:
submission = pd.DataFrame(prediction,index=test_data.index)
submission.rename(columns={0:"Cover_Type"},inplace=True)
submission["Id"] = test_data.Id
submission = submission[["Id","Cover_Type"]]
submission